This notebook is for the genre prediction task.

# Importation

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skmultilearn.problem_transform import BinaryRelevance

In [2]:
#TODO change here the path to your data folder where you stored the csv 'books_and_genres.csv'
data_path='./data'

In [3]:
#Importing the X and y
with open(data_path+'/X.pkl', 'rb') as f:
    X=pickle.load(f)

with open(data_path+'/y.pkl', 'rb') as f:
    y=pickle.load(f)

In [4]:
#Building our train test and validation set
X_pretrain, X_test, y_pretrain, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_pretrain, y_pretrain, test_size=0.2, random_state=42)

# Doc2Vec

In [5]:
# tagged data: each doc number is a label
tagged_data = [TaggedDocument(words=doc,
                              tags=[str(i)]) for i, doc in enumerate(X_train)]
# train the Doc2vec model
model = Doc2Vec(vector_size=20,
                min_count=2, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)

In [6]:
# get the document vectors
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# Model

### Without doc2vec pre-processing

In [7]:
def dummy(doc):
    return doc

def preprocess_train_valid(X_train, X_valid):
    vectorizer = CountVectorizer(
            tokenizer=dummy,
            preprocessor=dummy,
        ) 
    X_train_new=vectorizer.fit_transform(X_train)
    X_valid_new=vectorizer.transform(X_valid)
    return (X_train_new, X_valid_new)

In [8]:
X_train, X_valid=preprocess_train_valid(X_train, X_valid)

/Users/clea/opt/anaconda3/envs/opht/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
# Binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_valid)
# accuracy
print("Accuracy = ", accuracy_score(y_valid,predictions))

### With doc2vec pre-processing

In [ ]:
# Binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train_vec, y_train)
# predict
predictions = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ", accuracy_score(y_test,predictions))

Accuracy =  0.0
